In [17]:
from collections import deque
import heapq
import time

class Node:
    def __init__(self, state, parent=None, action=None, cost=0, heuristic=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = cost
        self.heuristic = heuristic

    def __lt__(self, other):
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

def get_neighbors(maze, position):
    row, col = position
    neighbors = []
    for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < len(maze) and 0 <= new_col < len(maze[0]) and maze[new_row][new_col] != '*':
            neighbors.append(((new_row, new_col), (dr, dc)))
    return neighbors

def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

## Breadth First Search

In [18]:
def bfs(maze, start, goal):
    frontier = deque([Node(start)])
    explored = set()
    
    while frontier:
        node = frontier.popleft()
        
        if node.state == goal:
            return node
        
        explored.add(node.state)
        
        for neighbor, action in get_neighbors(maze, node.state):
            if neighbor not in explored:
                child = Node(neighbor, node, action, node.cost + 1)
                frontier.append(child)
    
    return None

## Depth First Search


In [24]:
def dfs(maze, start, goal):
    frontier = [Node(start)]
    explored = set()
    
    while frontier:
        node = frontier.pop()
        
        if node.state == goal:
            return node
        
        explored.add(node.state)
        
        for neighbor, action in get_neighbors(maze, node.state):
            if neighbor not in explored:
                child = Node(neighbor, node, action, node.cost + 1)
                frontier.append(child)
    
    return None

## Uniform Cost Search

In [26]:
def uniform_cost_search(maze, start, goal):
    frontier = [(0, Node(start))]
    explored = set()
    
    while frontier:
        cost, node = heapq.heappop(frontier)
        
        if node.state == goal:
            return node
        
        if node.state in explored:
            continue
        
        explored.add(node.state)
        
        for neighbor, action in get_neighbors(maze, node.state):
            if neighbor not in explored:
                new_cost = cost + 1
                child = Node(neighbor, node, action, new_cost)
                heapq.heappush(frontier, (new_cost, child))
    
    return None

## Greedy Best First Search

In [27]:
def greedy_best_first_search(maze, start, goal):
    frontier = [(manhattan_distance(start, goal), Node(start))]
    explored = set()
    
    while frontier:
        _, node = heapq.heappop(frontier)
        
        if node.state == goal:
            return node
        
        explored.add(node.state)
        
        for neighbor, action in get_neighbors(maze, node.state):
            if neighbor not in explored:
                child = Node(neighbor, node, action)
                heapq.heappush(frontier, (manhattan_distance(neighbor, goal), child))
    
    return None

## A* Seacrh

In [25]:
def astar(maze, start, goal):
    frontier = []
    heapq.heappush(frontier, Node(start, heuristic=manhattan_distance(start, goal)))
    explored = set()
    
    while frontier:
        node = heapq.heappop(frontier)
        
        if node.state == goal:
            return node
        
        explored.add(node.state)
        
        for neighbor, action in get_neighbors(maze, node.state):
            if neighbor not in explored:
                cost = node.cost + 1
                heuristic = manhattan_distance(neighbor, goal)
                child = Node(neighbor, node, action, cost, heuristic)
                heapq.heappush(frontier, child)
    
    return None

## Beam Search

In [28]:
def beam_search(maze, start, goal, beam_width=2):
    frontier = [Node(start)]
    
    while frontier:
        next_frontier = []
        
        for node in frontier:
            if node.state == goal:
                return node
            
            for neighbor, action in get_neighbors(maze, node.state):
                child = Node(neighbor, node, action, node.cost + 1, manhattan_distance(neighbor, goal))
                next_frontier.append(child)
        
        frontier = sorted(next_frontier, key=lambda x: x.heuristic)[:beam_width]

## Maze problem

In [21]:
def solve_maze(maze, algorithm):
    start = None
    goal = None
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == 'M':
                start = (i, j)
            elif cell == 'C':
                goal = (i, j)
    
    if start is None or goal is None:
        return None
    
    start_time = time.time()
    solution = algorithm(maze, start, goal)
    end_time = time.time()
    
    if solution:
        path = []
        while solution:
            path.append(solution.state)
            solution = solution.parent
        return path[::-1], end_time - start_time
    else:
        return None, end_time - start_time

In [31]:
def read_maze_from_csv(file_path):
    maze = []
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            maze.append(row)
    return maze

def solve_maze(maze, algorithm):
    start = None
    goal = None
    for i, row in enumerate(maze):
        for j, cell in enumerate(row):
            if cell == 'M':
                start = (i, j)
            elif cell == 'C':
                goal = (i, j)
    
    if start is None or goal is None:
        return None
    
    start_time = time.time()
    solution = algorithm(maze, start, goal)
    end_time = time.time()
    
    if solution:
        path = []
        while solution:
            path.append(solution.state)
            solution = solution.parent
        return path[::-1], end_time - start_time
    else:
        return None, end_time - start_time
    
# Example usage
csv_file_path = './maze/maze_5x5.csv'  # Make sure this file is in the same directory as your script
maze = read_maze_from_csv(csv_file_path)

algorithms = [
    ('BFS', bfs),
    ('DFS', dfs),
    ('A*', astar),
    ('Uniform Cost Search', uniform_cost_search),
    ('Greedy Best-First Search', greedy_best_first_search)
]

for name, algorithm in algorithms:
    path, runtime = solve_maze(maze, algorithm)
    if path:
        print(f"{name} found a solution in {runtime:.6f} seconds:")
        print(f"Path length: {len(path)}")
        print(f"Path: {path}")
    else:
        print(f"{name} failed to find a solution in {runtime:.6f} seconds")
    print()

BFS found a solution in 0.000074 seconds:
Path length: 11
Path: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3)]

DFS found a solution in 0.000049 seconds:
Path length: 11
Path: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3)]

A* found a solution in 0.000086 seconds:
Path length: 11
Path: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3)]

Uniform Cost Search found a solution in 0.000050 seconds:
Path length: 11
Path: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3)]

Greedy Best-First Search found a solution in 0.000060 seconds:
Path length: 11
Path: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3)]



## N-Queens 

In [33]:
import csv
from collections import deque
import heapq
import time
import copy

class Node:
    def __init__(self, state, parent=None, action=None, cost=0, heuristic=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.cost = cost
        self.heuristic = heuristic

    def __lt__(self, other):
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

def read_csv(file_path):
    data = []
    with open(file_path, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            data.append(row)
    return data

def is_maze(data):
    return any('M' in row for row in data)

def get_maze_neighbors(maze, position):
    row, col = position
    neighbors = []
    for dr, dc in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < len(maze) and 0 <= new_col < len(maze[0]) and maze[new_row][new_col] != '*':
            neighbors.append(((new_row, new_col), (dr, dc)))
    return neighbors

def manhattan_distance(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_queens_neighbors(board):
    neighbors = []
    n = len(board)
    for col in range(n):
        for row in range(n):
            if board[row][col] == '_':
                new_board = copy.deepcopy(board)
                new_board[row][col] = 'Q'
                neighbors.append((new_board, (row, col)))
    return neighbors

def count_attacking_queens(board):
    n = len(board)
    count = 0
    for i in range(n):
        for j in range(n):
            if board[i][j] != '_':
                # Check row and column
                for k in range(n):
                    if k != j and board[i][k] != '_':
                        count += 1
                    if k != i and board[k][j] != '_':
                        count += 1
                # Check diagonals
                for k in range(1, n):
                    if i+k < n and j+k < n and board[i+k][j+k] != '_':
                        count += 1
                    if i-k >= 0 and j-k >= 0 and board[i-k][j-k] != '_':
                        count += 1
                    if i+k < n and j-k >= 0 and board[i+k][j-k] != '_':
                        count += 1
                    if i-k >= 0 and j+k < n and board[i-k][j+k] != '_':
                        count += 1
    return count // 2  # Divide by 2 because each pair is counted twice

def bfs(problem, start, goal):
    frontier = deque([Node(start)])
    explored = set()
    
    while frontier:
        node = frontier.popleft()
        
        if node.state == goal:
            return node
        
        explored.add(str(node.state))
        
        for neighbor, action in (get_maze_neighbors(problem, node.state) if is_maze(problem) else get_queens_neighbors(node.state)):
            if str(neighbor) not in explored:
                child = Node(neighbor, node, action, node.cost + 1)
                frontier.append(child)
    
    return None

def dfs(problem, start, goal):
    frontier = [Node(start)]
    explored = set()
    
    while frontier:
        node = frontier.pop()
        
        if node.state == goal:
            return node
        
        explored.add(str(node.state))
        
        for neighbor, action in (get_maze_neighbors(problem, node.state) if is_maze(problem) else get_queens_neighbors(node.state)):
            if str(neighbor) not in explored:
                child = Node(neighbor, node, action, node.cost + 1)
                frontier.append(child)
    
    return None

def astar(problem, start, goal):
    frontier = []
    heapq.heappush(frontier, Node(start, heuristic=manhattan_distance(start, goal) if is_maze(problem) else count_attacking_queens(start)))
    explored = set()
    
    while frontier:
        node = heapq.heappop(frontier)
        
        if node.state == goal:
            return node
        
        explored.add(str(node.state))
        
        for neighbor, action in (get_maze_neighbors(problem, node.state) if is_maze(problem) else get_queens_neighbors(node.state)):
            if str(neighbor) not in explored:
                cost = node.cost + 1
                heuristic = manhattan_distance(neighbor, goal) if is_maze(problem) else count_attacking_queens(neighbor)
                child = Node(neighbor, node, action, cost, heuristic)
                heapq.heappush(frontier, child)
    
    return None

def uniform_cost_search(problem, start, goal):
    frontier = [(0, Node(start))]
    explored = set()
    
    while frontier:
        cost, node = heapq.heappop(frontier)
        
        if node.state == goal:
            return node
        
        if str(node.state) in explored:
            continue
        
        explored.add(str(node.state))
        
        for neighbor, action in (get_maze_neighbors(problem, node.state) if is_maze(problem) else get_queens_neighbors(node.state)):
            if str(neighbor) not in explored:
                new_cost = cost + 1
                child = Node(neighbor, node, action, new_cost)
                heapq.heappush(frontier, (new_cost, child))
    
    return None

def greedy_best_first_search(problem, start, goal):
    frontier = [(manhattan_distance(start, goal) if is_maze(problem) else count_attacking_queens(start), Node(start))]
    explored = set()
    
    while frontier:
        _, node = heapq.heappop(frontier)
        
        if node.state == goal:
            return node
        
        explored.add(str(node.state))
        
        for neighbor, action in (get_maze_neighbors(problem, node.state) if is_maze(problem) else get_queens_neighbors(node.state)):
            if str(neighbor) not in explored:
                child = Node(neighbor, node, action)
                heapq.heappush(frontier, (manhattan_distance(neighbor, goal) if is_maze(problem) else count_attacking_queens(neighbor), child))
    
    return None

# def beam_search(problem, start, goal, beam_width=2):
#     frontier = [Node(start)]
    
#     while frontier:
#         next_frontier = []
        
#         for node in frontier:
#             if node.state == goal:
#                 return node
            
#             for neighbor, action in (get_maze_neighbors(problem, node.state) if is_maze(problem) else get_queens_neighbors(node.state)):
#                 heuristic = manhattan_distance(neighbor, goal) if is_maze(problem) else count_attacking_queens(neighbor)
#                 child = Node(neighbor, node, action, node.cost + 1, heuristic)
#                 next_frontier.append(child)
        
#         frontier = sorted(next_frontier, key=lambda x: x.heuristic)[:beam_width]
    
#     return None

def solve_problem(problem, algorithm):
    if is_maze(problem):
        start = None
        goal = None
        for i, row in enumerate(problem):
            for j, cell in enumerate(row):
                if cell == 'M':
                    start = (i, j)
                elif cell == 'C':
                    goal = (i, j)
    else:  # n-queens
        start = problem
        goal = [['Q' if cell != '_' else cell for cell in row] for row in problem]
        goal = [row for row in goal if 'Q' in row]  # Remove empty rows
    
    if start is None or goal is None:
        return None
    
    start_time = time.time()
    solution = algorithm(problem, start, goal)
    end_time = time.time()
    
    if solution:
        path = []
        while solution:
            path.append(solution.state)
            solution = solution.parent
        return path[::-1], end_time - start_time
    else:
        return None, end_time - start_time

# Example usage
maze_file_path = './maze/maze_5x5.csv'
queens_file_path = './n-queens/4queens.csv'

maze = read_csv(maze_file_path)
queens = read_csv(queens_file_path)

algorithms = [
    ('BFS', bfs),
    ('DFS', dfs),
    ('A*', astar),
    ('Uniform Cost Search', uniform_cost_search),
    ('Greedy Best-First Search', greedy_best_first_search),
]

problems = [
    ("Maze", maze),
    ("6-Queens", queens)
]

for problem_name, problem in problems:
    print(f"\nSolving {problem_name} problem:")
    for name, algorithm in algorithms:
        path, runtime = solve_problem(problem, algorithm)
        if path:
            print(f"{name} found a solution in {runtime:.6f} seconds:")
            print(f"Path length: {len(path)}")
            if problem_name == "Maze":
                print(f"Path: {path}")
            else:
                print("Final board state:")
                for row in path[-1]:
                    print(' '.join(row))
        else:
            print(f"{name} failed to find a solution in {runtime:.6f} seconds")
        print()


Solving Maze problem:
BFS found a solution in 0.000059 seconds:
Path length: 11
Path: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3)]

DFS found a solution in 0.000056 seconds:
Path length: 11
Path: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3)]

A* found a solution in 0.000051 seconds:
Path length: 11
Path: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3)]

Uniform Cost Search found a solution in 0.000044 seconds:
Path length: 11
Path: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3)]

Greedy Best-First Search found a solution in 0.000059 seconds:
Path length: 11
Path: [(0, 1), (1, 1), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (2, 3), (1, 3), (0, 3)]


Solving 6-Queens problem:


KeyboardInterrupt: 